In [34]:
import numpy as np
import time
from scipy.optimize import line_search

In [35]:
def vector_norm(vec: np.ndarray) -> float:
    return max(map(abs, vec))

In [36]:
def find_min(func, a, b, xk, dk, eps=1e-4):
    phi = (1 + 5 ** 0.5) / 2
    a1 = b - (b - a) / phi
    b1 = a + (b - a) / phi

    while abs(b1 - a1) > eps:
        if func(xk + a1 * dk) < func(xk + b1 * dk):
            b = b1
        else:
            a = a1

        a1 = b - (b - a) / phi
        b1 = a + (b - a) / phi

    return (a + b) / 2

In [37]:
def grad_desc_method(func, func_grad, x0, M, epsilon1=1e-5, epsilon2=1e-5):
    xk = np.array(x0, dtype=float)
    k = 0
    
    while k < M:
        grad_fk = func_grad(xk)
        
        if np.linalg.norm(grad_fk) < epsilon1:
            break
        
        dk = -grad_fk
        alpha = find_min(func, -1000, 1000, xk, dk)

        xk_new = xk + alpha * dk
        
        if np.linalg.norm(xk_new - xk) < epsilon1 and abs(func(xk_new) - func(xk)) < epsilon2:
            break
        
        xk = xk_new
        k += 1
        
    return xk, k
    

In [38]:
def fletch_reeves_method(func, func_grad, x0, M,  eps=1e-7):
    dk = -func_grad(x0)
    xk = x0.copy()
    k = 0
    while k < M:
        if vector_norm(func_grad(xk)) < eps:
            break

        alpha = find_min(func, -1000, 1000, xk, dk, 1e-4)
        if alpha > 1e-4:
            xk1 = xk + alpha * dk
        wk = (vector_norm(func_grad(xk1)) ** 2) / (vector_norm(func_grad(xk)) ** 2)
        dk = -func_grad(xk1) + wk * dk
        xk = xk1.copy()
        k += 1
    return xk, k

In [39]:
def dav_fletch_pow_method(func, func_grad, x0, M, epsilon1=1e-5, epsilon2=1e-5): 
    Gk = np.eye(len(x0))
    xk = np.array(x0, dtype=float)
    k = 0
    while k < M:
        if np.linalg.norm(func_grad(xk)) < epsilon1:
            break
        dk = np.dot(-Gk, func_grad(xk))

        alpha = find_min(func, -1000, 1000, xk, dk, 1e-4)
            
        xk1 = xk + alpha * dk
        delta_xk = xk1 - xk
        delta_gk = func_grad(xk1) - func_grad(xk)
        delta_Gk = (np.outer(delta_xk, delta_xk.T) / (delta_xk.T @ delta_gk)) - ((np.outer(Gk @ delta_gk, delta_gk.T @ Gk.T)) / (delta_gk.T @ Gk @ delta_gk))
        Gk = Gk + delta_Gk
        xk = xk1
        k += 1
    return xk, k

In [40]:
def rosenbrock_hessian(a, b, f0, n):
    def hessian_func(x):
        H = np.zeros((n, n))

        # Элементы вдоль главной диагонали
        for i in range(n):
            if i == 0:
                # Вторая производная по x0
                H[i, i] = 12 * a * x[0] ** 2 - 4 * a * x[1] + 2 * b
            elif i == n - 1:
                # Вторая производная по x_(n-1)
                H[i, i] = 2 * a
            else:
                # Вторая производная по x_i
                H[i, i] = 12 * a * x[i] ** 2 - 4 * a * x[i + 1] + 2 * b + 2 * a

        # Внедиагональные элементы (вторая производная по x_i и x_(i+1))
        for i in range(n - 1):
            H[i, i + 1] = -4 * a * x[i]
            H[i + 1, i] = H[i, i + 1]  # Симметричная матрица

        return H
    
    return hessian_func

In [41]:
def levenb_marq_method(func, func_grad, hessian, x0, M, mu0=1e4, epsilon1=1e-5, epsilon2=1e-5):
    xk = np.array(x0, dtype=float)
    n = len(xk)
    mu_k = mu0
    k = 0
    
    while k < M:
        grad_fk = func_grad(xk)
        
        if np.linalg.norm(grad_fk) < epsilon1:
            break
        
        Hk = hessian(xk)
        Hk_mu = Hk + mu_k * np.eye(n)
        Hk_mu_inv = np.linalg.inv(Hk_mu)
        
        dk = -np.dot(Hk_mu_inv, grad_fk)
        xk_new = xk + dk
        
        if func(xk_new) < func(xk):
            mu_k /= 2
        else:
            mu_k *= 2
        
        xk = xk_new
        k += 1
    
    return xk, k

In [42]:
def rosenbrock(a, b, f0, n):
    def rosenbroke_calc(x):
        return sum(a * (x[i] ** 2 - x[i + 1]) ** 2 + b * (x[i] - 1) ** 2 for i in range(n - 1)) + f0
    return rosenbroke_calc

In [43]:
def rosenbrock_grad(a, b, f0, n):
    def rosenbrock_grad_calc(x):
        grad = []
        df_dx0 = 4 * a * x[0] * (x[0] ** 2 - x[1]) + 2 * b * (x[0] - 1)
        grad.append(df_dx0)
        for i in range(1, n - 1):
            df_dxi = 4 * a * x[i] * (x[i] ** 2 - x[i + 1]) + 2 * b * (x[i] - 1) - 2 * a * (x[i - 1] ** 2 - x[i])
            grad.append(df_dxi)
        df_dxn = - 2 * a * (x[n - 2] ** 2 - x[n - 1])
        grad.append(df_dxn)
        return np.array(grad)
    return rosenbrock_grad_calc

In [44]:
def run_optimization(optimizer, func, func_grad, hessian, x0, M, *params):
    start_time = time.time()

    if optimizer.__name__ == "levenb_marq_method":
        result, iterations = optimizer(func, func_grad, hessian, x0, M, 1e4, *params)
    else:
        result, iterations = optimizer(func, func_grad, x0, M, *params)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    return result, iterations, elapsed_time

In [45]:
x0 = np.array([0.0, 0.0, 0.0, 0.0]) 
a, b, f0, n = 30, 2, 80, 4
M = 100000

methods = [
    ('Gradient descent', grad_desc_method),
    ('Fletcher-Reeves', fletch_reeves_method),
    ('Davidon-Fletcher-Powell', dav_fletch_pow_method),
    ('Levenberg-Marquardt', levenb_marq_method)
]

rosenbrock_func = rosenbrock(a, b, f0, n)
rosenbrock_grad_func = rosenbrock_grad(a, b, f0, n)
rosenbrock_hess_func = rosenbrock_hessian(a, b, f0, n)

for name, method in methods:
    result, iterations, time_taken = run_optimization(method, rosenbrock_func, rosenbrock_grad_func, rosenbrock_hess_func, x0, M)
    print(f"Метод: {name}")
    print(f"Результат: {result}")
    print(f"Кол-во итераций: {iterations}")
    print(f"Время выполнения: {time_taken:.5f} секунд\n")

Метод: Gradient descent
Результат: [0.99981063 0.99961019 0.99921726 0.99840295]
Кол-во итераций: 1279
Время выполнения: 0.71612 секунд
Метод: Fletcher-Reeves
Результат: [0.99775084 0.90773772 0.82901922 0.64253132]
Кол-во итераций: 100000
Время выполнения: 47.55164 секунд

Метод: Davidon-Fletcher-Powell
Результат: [1.         1.         1.00000001 1.00000003]
Кол-во итераций: 18
Время выполнения: 0.00821 секунд

Метод: Levenberg-Marquardt
Результат: [0.99999963 0.99999926 0.99999849 0.99999693]
Кол-во итераций: 20
Время выполнения: 0.00502 секунд


In [46]:
result, iterations, time_taken = run_optimization(fletch_reeves_method, rosenbrock_func, rosenbrock_grad_func, rosenbrock_hess_func, x0, M)
print('Fletcher Reeves')
print(f"Результат: {result}")
print(f"Кол-во итераций: {iterations}")
print(f"Время выполнения: {time_taken:.5f} секунд\n")

Fletcher Reeves
Результат: [0.99775084 0.90773772 0.82901922 0.64253132]
Кол-во итераций: 100000
Время выполнения: 48.71819 секунд
